In [1]:
import numpy as np
from sklearn.cluster import SpectralClustering,KMeans
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef,accuracy_score,precision_score,recall_score,confusion_matrix
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import os
import pandas as pd
from pathlib import Path

In [2]:
# HL: transfer from dataden "/umms-kinfai/duolin/ying/reditools2_candidates/"
hash_candidat={}
hash_candidat['AFG-H1_directRNA']='H1-AFG.candidate_sites.tab'
hash_candidat['AFG-H9_directRNA']='H9-AFG.candidate_sites.tab'
hash_candidat["PGC-H1_directRNA"]='H1-PGC.candidate_sites.tab'
hash_candidat["DE-H1_directRNA"]='H1-DE.candidate_sites.tab'
hash_candidat["DE-H9_directRNA"]='H9-DE.candidate_sites.tab'
hash_candidat["GM12878_directRNA"]='GM12878.candidate_sites.tab'
hash_candidat["H1-hESC_directRNA"]='H1-hESC.candidate_sites.tab'
hash_candidat["H9-hESC_directRNA"]='H9-hESC.candidate_sites.tab'
hash_candidat['HEK293T_DKO_directRNA']='HEK293T_WT.candidate_sites.tab'
hash_candidat["HEK293T_WT_directRNA"]='HEK293T_WT.candidate_sites.tab'
hash_candidat["HEK_WT_pass"]='HEK293T_WT.candidate_sites.tab'


In [3]:
modelist=[
    'hg38_incre5train__HEK293T_WT_directRNA',
    'hg38_incre5train__HEK293T_WT_directRNA_AFG-H1_directRNA',
    'hg38_incre5train__HEK293T_WT_directRNA_H9-hESC_directRNA', #no noncandidate data
'hg38_incre5train__HEK293T_WT_directRNA_AFG-H1_directRNA_DE-H1_directRNA', #no noncandidate data
'hg38_incre5train__HEK293T_WT_directRNA_AFG-H1_directRNA_DE-H1_directRNA_H9-hESC_directRNA', #no data

'hg38_merge9alldata5_noearlystop_run1ep40_run2ep60_negsite_KOnonw5_withLSTM',#second general model
]

modelnames=['HEK293T','HEK293T;H1-PE','HEK293T;H9-hESC','HEK293T;H1-PE;H1-DE','HEK293T;H1-PE;H1-DE;H9-hESC','all']

In [4]:
AG_ratio_per_site={}
shortreadcoverage={}
datatype='H1-hESC_directRNA'
long_reads_min_coverage= 5

candidatefile="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2a/reditools2_candidates/"+hash_candidat[datatype]

input = open(candidatefile,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1]
    chrpos=chr_+"-"+pos_
    AG_ratio_per_site[chrpos]=float(line.split("\t")[3])
    shortreadcoverage[chrpos]=float(line.split("\t")[4])

In [5]:
num_bins = 5
binsize=1/num_bins
def binize_list(true_list,predict_list):
    true_list,predict_list = np.array(true_list),np.array(predict_list)
    ratio_list = np.arange(0,1,binsize) #np.random.uniform(0.0, 1.0, 30)
    ratio_list.sort()
    sample_size=[]
    AG_ratios_bin=[]
    predict_ratio_bin = []
    ratio_bin_list=[]
    for index in range(len(ratio_list)):
        start = ratio_list[index]
        end = start+binsize
        index_start_end = np.where((true_list<=end) &(true_list>start))[0]
        if(len(index_start_end)>1):
            ratio_bin_list.append(start)
            AG_ratios_bin.append(true_list[index_start_end])
            predict_ratio_bin.append(predict_list[index_start_end])
            sample_size.append(len(predict_list[index_start_end]))
    return AG_ratios_bin,predict_ratio_bin,sample_size


In [7]:
performance_dict = {}
mae_dict = {}
for model,modelname in zip(modelist,modelnames):
    filename="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3d/ReDD_results/"+model+"/"+"test_"+datatype+"_onlycandidate.txt"
    input = open(filename)
    coverage = {}
    pos_coverage = {}
    cutoff = 0.5
    for line in input:
        score=float(line.split("\t")[-1])
        if score>=cutoff:
           predict_label=1
        else:
           predict_label=0
        transid= line.split("\t")[2]
        transpos = line.split("\t")[3]
        chrpos = transid+"-"+transpos
        if chrpos not in pos_coverage.keys():
            pos_coverage[chrpos]=predict_label
            coverage[chrpos]=1
        else:
            pos_coverage[chrpos]+=predict_label
            coverage[chrpos]+=1
    predict_value_all = {}
    true_value_all = {}
    use_coverage_cutoff=1
    for site in AG_ratio_per_site:   
         if site in pos_coverage:
            if coverage[site] >= long_reads_min_coverage:
                predict_value_all[site]=float(pos_coverage[site]/coverage[site])
                true_value_all[site] = AG_ratio_per_site[site]
    # export to csv
    df = pd.DataFrame({'truth':true_value_all,'ReDD':predict_value_all})
    df.index.name = 'Site'
    Path('plot_data/site_ratios/').mkdir(exist_ok=True,parents=True)
    df.to_csv(f'plot_data/site_ratios/{modelname}_site_ratio.tsv',sep='\t',index=True)
    mae_dict[modelname] = df
    true_list=[]
    predict_list = []
    coords_list = []
    for key in predict_value_all:#1259
        true_list.append(true_value_all[key])
        predict_list.append(predict_value_all[key])
        coords_list.append(key)
    predict_list=np.asarray(predict_list)
    true_list=np.asarray(true_list)
    coords_list = np.asarray(coords_list)
    print(np.mean(np.abs(predict_list-true_list)))
    # binize
    AG_ratios_bin,predict_ratio_bin,sample_size = binize_list(true_list,predict_list)
    
    # calculate MAE
    mae=np.abs(np.asarray([np.median(x) for x in AG_ratios_bin])-np.asarray([np.median(x) for x in predict_ratio_bin])).mean()    
    print(mae)
    print(modelname)
    print('----')
#     pcc=pearsonr([np.median(x) for x in AG_ratios_bin], [np.median(x) for x in predict_ratio_bin])[0]
    performance_dict[modelname] = [mae]

0.0894658500278209
0.2425844322175109
HEK293T
----
0.115938374369105
0.17315292158867085
HEK293T;H1-PE
----
0.13023371087225577
0.11823605651226576
HEK293T;H9-hESC
----
0.15909163162355963
0.10457340685716372
HEK293T;H1-PE;H1-DE
----
0.16602358067262202
0.08502062182556239
HEK293T;H1-PE;H1-DE;H9-hESC
----
0.1714074200604227
0.06600349673921693
all
----


In [38]:
performance_dict

{'HEK293T': [0.2425844322175109],
 'HEK293T;H1-PE': [0.17315292158867085],
 'HEK293T;H9-hESC': [0.11823605651226576],
 'HEK293T;H1-PE;H1-DE': [0.10457340685716372],
 'HEK293T;H1-PE;H1-DE;H9-hESC': [0.08502062182556239],
 'all': [0.06600349673921693]}

In [7]:
df = pd.DataFrame(performance_dict).T
df.index.name ='Model'
df.columns=['MAE']

In [36]:
df.to_csv('plot_data/quantification_metrics.tsv',sep='\t')

In [87]:
import scipy.stats
def compare_df(metA,metB):
    metA_error = np.abs(mae_dict[metA]['ReDD'] - mae_dict[metA]['truth'])
    metB_error = np.abs(mae_dict[metB]['ReDD'] - mae_dict[metB]['truth'])
    error_df = pd.DataFrame({'truth':mae_dict[metA]['truth'],'metA':metA_error,'metB':metB_error})
    return error_df,scipy.stats.wilcoxon(error_df['metB'],error_df['metA'],alternative='greater')

In [88]:
metA = 'HEK293T'
metB = 'HEK293T;H1-PE'
error_df,p = compare_df(metA,metB)
print(p)

WilcoxonResult(statistic=826136.0, pvalue=1.1263426472566365e-44)


In [89]:
metA = 'HEK293T'
metB = 'HEK293T;H1-PE;H1-DE'
error_df,p = compare_df(metA,metB)
print(p)

WilcoxonResult(statistic=1555188.0, pvalue=4.7644916036956485e-124)


In [90]:
metA = 'HEK293T'
metB = 'HEK293T;H9-hESC'
error_df,p = compare_df(metA,metB)
print(p)

WilcoxonResult(statistic=1106224.5, pvalue=2.700387794547009e-79)


In [91]:
metA = 'HEK293T;H1-PE;H1-DE;H9-hESC'
metB = 'all'
error_df,p = compare_df(metA,metB)
print(p)

WilcoxonResult(statistic=965143.0, pvalue=0.5568496389326504)


In [47]:
ps = []
for start,end in zip([0,0.2,0.4,0.6,0.8],[0.2,0.4,0.6,0.8,1.0]):
    new_error_df = error_df[(error_df['truth'] > start) & (error_df['truth'] <= end)]
    p = scipy.stats.wilcoxon(new_error_df['metB'],new_error_df['metA'],alternative='less').pvalue
    ps.append(p)

In [48]:
np.array(ps)/len(ps)

array([2.00000000e-01, 6.28117787e-02, 3.40829517e-03, 1.32893651e-07,
       2.33229334e-05])

In [49]:
import statsmodels.stats.multitest

In [50]:
statsmodels.stats.multitest.fdrcorrection(ps, alpha=0.05, method='indep', is_sorted=False)

(array([False, False,  True,  True,  True]),
 array([1.00000000e+00, 3.92573617e-01, 2.84024597e-02, 3.32234127e-06,
        2.91536668e-04]))